In [41]:
import numpy as np
import pandas as pd
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

# Import SQL Data


The data for plotting NDSI changes is loaded from an SQL file `Visualization_data_sql.db`, from which the tables:

* `NDSI_location` - The data where the Snow Cover Index (NDSI) is indexed by year, month of observation, and geographic coordinates for visualization on a map.
* `NDSI_by_years_lineplot_data` - The data containing descriptive statistics of NDSI by observation date (year, month) for each altitude range above sea level for drawing line plots.




In [ ]:
!wget 'https://www.dropbox.com/scl/fi/u7fuf81576hdjmq3ql5up/Visualization_data_sql.db?rlkey=qic0kyfv3tfs6qg4drjdfpwas&dl=0' -O Visualization_data_sql.db

In [ ]:
%cd /content
conn = sqlite3.connect('Visualization_data_sql.db')
query = f"SELECT * FROM NDSI_location"
data_full = pd.read_sql_query(query, conn)
conn.close()

# Mapbox by months.

Below in the dropdown list, you can select the observation month. From the dataset (from the table NDSI_location), the data for each year from 2005 to the current year will be filtered out only for the selected month. Then you can plot the graph, where the slider will help you display NDSI on the map for the selected year. If you change the month value, you will need to run the plot code again.

In [48]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
def filter_dataframe(month: str)->None:
  global data_month
  data_month = data_full[data_full.month == month]
  data_month.reset_index(drop=True, inplace=True)
  data_month.drop([i for i in range(0, data_month.shape[0], 4)], axis=0, inplace=True)

month_dropdown = widgets.Dropdown(
    options=months,
    description='Select Month:',
    disabled=False
    )

def on_month_change(change)->None:
    filter_dataframe(month_dropdown.value)

month_dropdown.observe(on_month_change, names='value')
display(month_dropdown)
filter_dataframe(month_dropdown.value)

Dropdown(description='Select Month:', options=('January', 'February', 'March', 'April', 'May', 'June', 'July',…

In [ ]:
fig = px.density_mapbox(data_month, lat='lat', lon='lon', z='NDSI', radius=14, zoom=7, opacity=.7, color_continuous_scale='GnBu',
                        center=dict(lat=41.931586, lon=74.582786), mapbox_style="open-street-map", animation_frame="year")
fig.update_layout(width=1600, height=1200)
fig.show()

# Mapbox by years.

Below in the dropdown list, you can select the observation year. From the dataset (from the table NDSI_location), the data for each month will be filtered out only for the selected year. Then you can plot the graph, where the slider will help you display NDSI on the map for the selected month. If you change the year value, you will need to run the plot code again.

In [90]:
years = data_full.year.unique().tolist()
def filter_dataframe(year: int)->None:
  global data_year
  data_year = data_full[data_full.year == year]

year_dropdown = widgets.Dropdown(
    options=years,
    description='Select Year:',
    disabled=False
    )

def on_year_change(change)->None:
    filter_dataframe(year_dropdown.value)

year_dropdown.observe(on_year_change, names='value')
display(year_dropdown)
filter_dataframe(year_dropdown.value)

Dropdown(description='Select Year:', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015…

In [ ]:
fig = px.density_mapbox(data_year, lat='lat', lon='lon', z='NDSI', radius=12, zoom=7, opacity=.7, color_continuous_scale='GnBu',
                        center=dict(lat=41.931586, lon=74.582786), mapbox_style="open-street-map", animation_frame="month", )
fig.update_layout(width=1600, height=1200)
fig.show()

# Line plots for monthly averaged NDSI values.

Let's load the data from the table `NDSI_by_years_lineplot_data`, which includes the following features:

* `Month` - Month for which NDSI observations were averaged.
* `mean_NDSI` - Average NDSI for the month over the specified altitude range.
* `Elevation` - Altitude range in meters above sea level.
* `year` - Year of observation.

The plots below depict the relationship between the averaged NDSI and the time stamp (Month or Year), where the slider sets the selected year or observation month. The NDSI values are also grouped by altitude range.

In [56]:
%cd /content
conn = sqlite3.connect('Visualization_data_sql.db')
query = f"SELECT * FROM NDSI_by_years_lineplot_data"
data_ndsi_line = pd.read_sql_query(query, conn)
conn.close()

In [58]:
fig = px.line(data_ndsi_line, x="Month", y="mean_NDSI", color='Elevation', markers=True, title='Average NDSI',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f', '#042e00'], animation_frame="year")
fig.show()

In [59]:
fig = px.line(data_ndsi_line, x="year", y="mean_NDSI", color='Elevation', markers=True, title='Average NDSI',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f', '#042e00'], animation_frame="Month", range_y=(-1, 90))
fig.show()

# Multi Feature Plot

In this point, we will plot not only the average NDSI values. We will show the minimum, mean, median, and quartile values of the NDSI for each month from 2005 to the present. So we need to prepare the corresponding dataframe.

In [61]:
data_multi_feat = pd.DataFrame(columns = ['Month', 'Elevation', 'NDSI', 'Aggr_func', 'year'])
col = ['mean_NDSI', 'min_NDSI', 'max_NDSI', 'median_NDSI', 'qu_25_NDSI', 'qu_75_NDSI']
func = ['Mean', 'Min', 'Max', 'Median', 'QU_25', 'QU_75']

for c, f in zip(col, func):
  df_temp = pd.DataFrame({'Month': data_ndsi_line.Month,
                          'Elevation': data_ndsi_line.Elevation,
                          'year': data_ndsi_line.year,
                          'NDSI': data_ndsi_line[c],
                          'Aggr_func': f})
  data_multi_feat = pd.concat([data_multi_feat, df_temp], axis=0)

data_multi_feat['date'] = data_multi_feat.Month.astype(str) + ', ' + data_multi_feat.year.astype(str)
data_multi_feat.date = pd.to_datetime(data_multi_feat.date)
data_multi_feat = data_multi_feat.sort_values('date')
elev_type_order=['500-1000', '1000-2000', '2000-3000', '3000-4000', '4000-5000', '5000-6000', '6000-7000']
data_multi_feat['Elevation'] = pd.Categorical(data_multi_feat['Elevation'], categories=elev_type_order, ordered=True)
data_multi_feat = data_multi_feat.sort_values(by=['Elevation', 'date'])

## Plot

The plot shows the NDSI value for the specified altitude range, aggregated by the following statistical functions:


* `Min` - Minimum NDSI observed for the month
* `QU_25` - 25th percentile NDSI observed for the month
* `Median` - Median NDSI observed for the month
* `QU_75` - 75th percentile NDSI observed for the month
* `Mean` - Mean (Average) NDSI observed for the month
* `Max` - Maximum NDSI observed for the month

In [63]:
fig = px.line(data_multi_feat, x="date", y="NDSI", color='Aggr_func', markers=True, title='NDSI',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f'], animation_frame="Elevation", range_y=(0, 100))
fig.show()

# Plotting with a selectable time range.

In [86]:
years = data_multi_feat.year.unique().tolist()
def filter_dataframe(year_from: int, year_to: int)->None:
  global data_multi_feat_year
  data_multi_feat_year = data_multi_feat[(data_multi_feat.year >= year_from) & (data_multi_feat.year <= year_to)]
  func_order = ['Min', 'QU_25', 'Median', 'QU_75', 'Mean', 'Max'][::-1]
  data_multi_feat_year['Aggr_func'] = pd.Categorical(data_multi_feat_year['Aggr_func'], categories=func_order, ordered=True)
  data_multi_feat_year = data_multi_feat_year.sort_values(by=['date', 'Aggr_func'])

year_from_dropdown = widgets.Dropdown(
    options=years,
    description='Year From:',
    disabled=False
    )
year_to_dropdown = widgets.Dropdown(
    options=years,
    description='Year To:',
    disabled=False
    )

def on_year_change(change)->None:
    filter_dataframe(year_from_dropdown.value, year_to_dropdown.value)

year_from_dropdown.observe(on_year_change, names='value')
year_to_dropdown.observe(on_year_change, names='value')
display(year_from_dropdown, year_to_dropdown)
filter_dataframe(year_from_dropdown.value, year_to_dropdown.value)

Dropdown(description='Year From:', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, …

Dropdown(description='Year To:', options=(2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 20…

In [93]:
fig = px.line(data_multi_feat_year, x="date", y="NDSI", color='Aggr_func', markers=True, title=f'NDSI from {year_from_dropdown.value} to {year_to_dropdown.value} years.',
              color_discrete_sequence=["#6805f2", "#0521f2", "#027fde", "#04adbf", "#02a38b", '#007a2f'], animation_frame="Elevation", range_y=(-5, 100))
fig.show()